# Overview Materi

Jelaskan perbedaan singkat antara grid, randomized, bayesian search cv dengan optuna menurut pemahamanmu

source: https://www.youtube.com/watch?v=t-INgABWULw

# Import Data & Libraries

In [1]:
# jalankan hanya sekali
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 5.6 MB/s eta 0:00:00


In [2]:
# import library yang dibutuhkan di sini
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import optuna
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [64]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# Data Preprocessing

In [65]:
# ubah variabel kategorik ke numerik
df['species_num'] = df['species'].map({
    'setosa': 0,
    'versicolor': 1,
    'virginica': 2
})
subset_df = df[['sepal_length', 'species_num']]

In [66]:
# subsetting peubah
X = df.drop(['species'], axis=1)
y = df['species']

# Dataset Splitting

In [67]:
# split dengan rasio 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# Base Model Random Forest

In [68]:
# gunakan random forest classifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [70]:
y_pred = rfc.predict(X_test)

In [71]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(classification_report(y_test, y_pred))

Accuracy: 1.000
Precision: 1.000
Recall: 1.000
F1 Score: 1.000
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



# Optuna

In [76]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500, 10)
    max_depth = int(trial.suggest_float('max_depth', 10, 100, log=True))
    min_samples_split = int(trial.suggest_float('min_samples_split', 2, 10, log=True))
    min_samples_leaf = int(trial.suggest_float('min_samples_leaf', 1, 4, log=True))

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    score = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

    return score.mean()

Hyperparameter dapat disesuaikan dengan algoritma yang digunakan. Kali ini kita menggunakan Random Forest sehingga yang dapat kita select adalah *n_estimators, max_depth, min_samples_split,* dan *min_samples_leaf*

In [83]:
study = optuna.create_study(direction='maximize')

[I 2025-10-04 16:49:47,602] A new study created in memory with name: no-name-ac169762-60df-4230-8396-f3af5bc0d918


In [84]:
study.optimize(objective, n_trials=100)

/tmp/ipython-input-3817744718.py:2: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
Positional arguments ['self', 'name', 'low', 'high', 'step', 'log'] in suggest_int() have been deprecated since v3.5.0. They will be replaced with the corresponding keyword arguments in v5.0.0, so please use the keyword specification instead. See https://github.com/optuna/optuna/releases/tag/v3.5.0 for details.
  n_estimators = trial.suggest_int('n_estimators', 100, 500, 10)
/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:978: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_scorer.

KeyboardInterrupt: 

it may take a while... so just wait n see ^^
<br>
they recommend to set n_trials at 100 cz it seems there's no significant score increase after 100 trials (also inefficient too, you'll have to wait in a quite long time)

In [81]:
study.best_params

ValueError: No trials are completed yet.

Berikut hasil hyperparameter tuning dari Optuna

In [47]:
# cek hasil hyperparameter tuning dari Optuna
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'n_estimators': 350, 'max_depth': 94.10899861619659, 'min_samples_split': 5.458015499361727, 'min_samples_leaf': 1.2132056655040657}


# Random Forest Using Optuna

In [48]:
# simpan hasil best hyperparameter tuning ke variabel baru
best_score = study.best_value
print(f"Best Accuracy: {best_score:.3f}")

Best Accuracy: -0.064


In [ ]:
best_model = RandomForestClassifier(n_estimators=best_n_estimators,
                                   max_depth=best_max_depth,
                                   min_samples_split=best_min_samples_split,
                                   min_samples_leaf=best_min_samples_leaf)

best_model.fit(X_train, y_train)

In [52]:
y_pred = best_model.predict(X_test)

NotFittedError: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [53]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(classification_report(y_test, y_pred))

ValueError: continuous is not supported

Tidak terdapat kenaikan skor dengan sebelum menggunakan Optuna sebab skor yang dihasilkan melalui base model saja sudah bagus